<a href="https://colab.research.google.com/github/Shreyas911/PINN/blob/main/PINN_himalayan_mountain_glacier_fake_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchtext==0.10.0 -f https://download.pytorch.org/whl/cu111/torch_stable.html

### Import the required libraries
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.axes_grid1 import make_axes_locatable
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_xla ## Specially for TPUs
import torch_xla.core.xla_model as xm ## Specially for TPUs

     |████████████████████████████████| 149.9 MB 26 kB/s 
     |████████████████████████████████| 57 kB 2.7 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.293 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.
Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html
     |█████████████                   | 834.1 MB 1.5 MB/s eta 0:13:38tcmalloc: large alloc 1147494400 bytes == 0x557621816000 @  0x7fd722160615 0x5575e8cae4cc 0x5575e8d8e47a 0x5575e8cb12ed 0x5575e8da2e1d 0x5575e8d24e99 0x5575e8d1f9ee 0x5575e8cb2bda 0x5575e8d24d00 0x5575e8

## Specify the device

We now specify the device type. There are two options to leverge the power of parallel computing. We can either use GPUs or TPUs. In this case, we use TPUs. 

In [ ]:
### GPU ###
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#torch.cuda.empty_cache()
###########

### TPU ###
device = xm.xla_device()
###########

device

In [ ]:
class physics_informed_NN(nn.Module):
    
    def __init__ (self):

        super(physics_informed_NN, self).__init__()
        ### 3 inputs - x, z, t
        self.fc1 = nn.Linear(3, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 200)
        self.fc4 = nn.Linear(200, 200)
        self.fc5 = nn.Linear(200, 200)
        self.fc6 = nn.Linear(200, 200)
        self.fc7 = nn.Linear(200, 200)
        self.fc8 = nn.Linear(200, 1)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        x = torch.tanh(self.fc5(x))
        x = torch.relu(self.fc6(x))
        x = torch.tanh(self.fc7(x))
        x = self.fc8(x)
        return x

physics_informed_NN